In [1]:
# Install Tensorflow
!pip install -q tensorflow==2.0.0-beta1

     |████████████████████████████████| 87.9MB 53kB/s 
     |████████████████████████████████| 3.1MB 57.5MB/s 
     |████████████████████████████████| 501kB 48.8MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.0.0b1 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.


In [2]:
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

2.0.0-beta1


In [0]:
from tensorflow.keras.layers import Input, SimpleRNN, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
# Things you should automatically know and have memorized
# N = number of samples
# T = sequence length
# D = number of input features
# M = number of hidden units
# K = number of output units

In [0]:
#  Make some data
N = 1
T = 10
D = 3
K = 2
X = np.random.randn(N, T, D)

In [6]:
X

array([[[-2.09377265, -0.2686418 , -0.71358937],
        [ 0.02333655, -0.50280647, -1.5881825 ],
        [-0.7626657 , -0.34601755, -0.60874659],
        [ 0.70888101, -1.29597644, -0.51155012],
        [ 2.02789586,  0.72193897, -1.43007477],
        [-1.82169351,  1.28599829, -0.02309979],
        [-0.80220176, -1.78700252, -0.55316512],
        [-0.04834869, -0.49775731, -1.39775   ],
        [ 1.87786435,  0.02561037, -0.0074969 ],
        [ 0.42725083, -0.32696798,  0.79015044]]])

In [7]:
X.shape

(1, 10, 3)

In [0]:
# Make an RNN
M = 5 # number of hidden units
i = Input(shape=(T,D))
x = SimpleRNN(M)(i)
x = Dense(K)(x)

model = Model(i, x)

In [9]:
# Get the output 
Yhat = model.predict(X)
print(Yhat)

[[-0.5585815  0.9350939]]


In [10]:
# See if we can replicate this output
# Get the weights first
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 3)]           0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 5)                 45        
_________________________________________________________________
dense (Dense)                (None, 2)                 12        
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [11]:
# See whats returned 
model.layers[1].get_weights()

[array([[-0.41595876, -0.44751766, -0.00378144, -0.67950153, -0.3664265 ],
        [-0.06879222, -0.39235458, -0.20741868, -0.36169118, -0.7318173 ],
        [ 0.864829  ,  0.56557494, -0.84153396,  0.5626895 , -0.37583935]],
       dtype=float32),
 array([[ 0.11685073,  0.15892322, -0.22092476,  0.3500396 , -0.888681  ],
        [-0.49266502, -0.79795295, -0.31965914,  0.10383517, -0.08711162],
        [ 0.5445842 , -0.1443076 , -0.38204628,  0.6226063 ,  0.3860121 ],
        [-0.24461547, -0.06218129,  0.73127854,  0.6331955 ,  0.02432868],
        [-0.6222688 ,  0.55975276, -0.41022265,  0.279491  ,  0.2303487 ]],
       dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32)]

In [12]:
# Check their shapes
# Should make sense 
# First output is input > hidden
# Second output is hidden > hidden
# Third output is bias term (vector of length M)

a, b, c = model.layers[1].get_weights()
print(a.shape, b.shape, c.shape)

(3, 5) (5, 5) (5,)


In [0]:
Wx, Wh, bh = model.layers[1].get_weights()
Wo, bo = model.layers[2].get_weights()

In [15]:
h_last = np.zeros(M) # initial hidden state
x = X[0] # the one and only sample 
Yhats = [] # where we store the outputs

for t in range(T):
  h = np.tanh(x[t].dot(Wx) + h_last.dot(Wh) + bh)
  y = h.dot(Wo) + bo # we only care about this value on the last iteration
  Yhats.append(y)

  # important: assign h to h_last
  h_last = h

# print the final output
print(Yhats[-1])

[-0.55858136  0.93509384]
